In [1]:
import pandas as pd
import numpy as np
pd.set_eng_float_format(accuracy=3, use_eng_prefix=False)

In [2]:
raw_data = pd.read_csv('raw_data.csv', index_col=0)

# 1

In [3]:
def covariance(data, horizon = 1):
    names = list(data.columns.values)
    N = len(data)
    X = np.column_stack([data])
    X -= X.mean(axis=0) 
    cov = np.dot(X.T, X.conj()) / N
    if horizon == 260:
        cov = cov * 260   
    cov = pd.DataFrame(cov, index=names, columns=names)
    return cov

def rolling_cov(data, min_day = 260, max_day = 2600, horizon = 1):
    data = data.dropna()
    if len(data) < min_day:
        print('note enough data')
        return 0
    else:
        cov = []
        for i in range(len(data)):
            if i < (max_day - min_day):
                new = covariance(data.ix[:i+min_day], horizon)
            else:
                new = covariance(data.ix[(i+min_day-max_day):i+min_day], horizon)
            cov.append(new)
        cov = pd.Series(cov, index = data.index)
    return cov

In [4]:
N_cov_mat = rolling_cov(raw_data)

In [5]:
N_cov_mat[0]

,FX_MXN,FX_SGD,FX_ZAR,FX_INR,FX_BRL,FX_TRY
FX_MXN,24.731E-06,-2.539E-06,227.782E-09,-94.498E-09,23.929E-06,10.689E-06
FX_SGD,-2.539E-06,7.340E-06,5.300E-06,275.107E-09,716.767E-09,-878.357E-09
FX_ZAR,227.782E-09,5.300E-06,111.054E-06,865.600E-09,16.562E-06,22.571E-06
FX_INR,-94.498E-09,275.107E-09,865.600E-09,494.696E-09,335.543E-09,-272.049E-09
FX_BRL,23.929E-06,716.767E-09,16.562E-06,335.543E-09,316.960E-06,29.580E-06
FX_TRY,10.689E-06,-878.357E-09,22.571E-06,-272.049E-09,29.580E-06,110.137E-06


# 2

In [6]:
N_cov_mat.index

Index(['2002-01-01 00:00:00', '2002-01-02 00:00:00', '2002-01-03 00:00:00',
       '2002-01-04 00:00:00', '2002-01-07 00:00:00', '2002-01-08 00:00:00',
       '2002-01-09 00:00:00', '2002-01-10 00:00:00', '2002-01-11 00:00:00',
       '2002-01-14 00:00:00',
       ...
       '2015-12-18 00:00:00', '2015-12-21 00:00:00', '2015-12-22 00:00:00',
       '2015-12-23 00:00:00', '2015-12-24 00:00:00', '2015-12-25 00:00:00',
       '2015-12-28 00:00:00', '2015-12-29 00:00:00', '2015-12-30 00:00:00',
       '2015-12-31 00:00:00'],
      dtype='object', length=3653)

In [7]:
def cov_to_corr(cov_mat):
    D = np.sqrt(np.diag(cov_mat))
    corr = cov_mat/(np.asmatrix(D).T * np.asmatrix(D))
    return corr

In [8]:
def decomp(N_cov_mat):
    corr = []
    vol = []
    for i in N_cov_mat:
        i_vol = np.diag(i)
        vol.append(i_vol)
        corr.append(cov_to_corr(i))
    corr = pd.Series(corr, index = N_cov_mat.index)
    vol = pd.Series(vol, index = N_cov_mat.index)
    return corr, vol

In [9]:
corr, vol = decomp(N_cov_mat)

In [10]:
def vol_floor(N_cov_mat, vol_floor):
    N_corr, N_vol = decomp(N_cov_mat)
    New_N_cov = []
    for i in N_vol.index:
        vol = N_vol.ix[i]
        vol.flags.writeable = True
        vol[vol<vol_floor] = vol_floor
        New_N_cov.append(np.dot(np.asmatrix(vol).T , np.asmatrix(vol)) * N_corr.ix[i])
    New_N_cov = pd.Series(New_N_cov, index = N_cov_mat.index)
    return New_N_cov

In [11]:
New_N_cov = vol_floor(N_cov_mat, 0.003)

In [12]:
weight = [0.3, 0.2, 0.1, 0.1, -0.2, -0.3]

In [13]:
def ex_ante_vol(N_cov_mat, weight):
    weight = np.asmatrix(weight)
    N_ex_ante_ext_vol = []
    for cov_mat in N_cov_mat:
        ex_ante_ext_vol = weight * np.asmatrix(cov_mat) * weight.T
        N_ex_ante_ext_vol.append(ex_ante_ext_vol.item())
    N_ex_ante_ext_vol = pd.Series(N_ex_ante_ext_vol,index=N_cov_mat.index)
    return N_ex_ante_ext_vol

In [14]:
ex_ante_vol(N_cov_mat, weight).ix[0]

0.00083673206327745464

In [15]:
def asset_betas(N_cov_mat, weight):
    corr, vol = decomp(N_cov_mat)
    N_ex_ante_ext_vol = ex_ante_vol(N_cov_mat, weight)
    beta = []
    for i in N_cov_mat.index:
        beta_ = np.asmatrix(weight) * np.asmatrix(N_cov_mat.ix[i]) / N_ex_ante_ext_vol.ix[i]
        beta.append(beta_.tolist()[0])
    beta = pd.Series(beta, index=N_cov_mat.index)
    return beta

In [16]:
beta = asset_betas(N_cov_mat, weight)

In [17]:
len(beta)

3653

In [18]:
def mcr(N_cov_mat, weight):
    MCR = []
    for i in N_cov_mat.index:
        numerator = np.asmatrix(N_cov_mat.ix[i]) * np.asmatrix(weight).T
        denominator = np.asmatrix(weight) * numerator 
        MCR_ = np.array(weight) * np.array(numerator.T) * denominator.item()
        MCR.append(MCR_)
    MCR = pd.Series(MCR, index=N_cov_mat.index)
    return MCR

In [19]:
MCR = mcr(N_cov_mat, weight)

In [20]:
print(MCR.head())

2002-01-01 00:00:00    [[2.23787289292e-07, 1.00393787581e-07, 2.4359...
2002-01-02 00:00:00    [[2.23729914672e-07, 1.00383771181e-07, 2.4360...
2002-01-03 00:00:00    [[2.23741374361e-07, 1.00385372414e-07, 2.4364...
2002-01-04 00:00:00    [[2.23738062432e-07, 1.00371035402e-07, 2.4369...
2002-01-07 00:00:00    [[2.23780209712e-07, 1.00381143783e-07, 2.4355...
dtype: object


# 3

In [21]:
def covariance_vol_decay(data, decay = 0.1, horizon = 1):
    names = list(data.columns.values)
    time = np.arange(len(data))[::-1]
    decay = np.power((1 - decay), time)
    X = np.column_stack([data])
    X -= X.mean(axis=0) 
    cov = np.dot((X.T * decay), X.conj()) / decay.sum()
    if horizon == 260:
        cov = cov * 260
    cov = pd.DataFrame(cov, index=names, columns=names)
    return cov

def rolling_cov_vol_decay(data, decay = 0.1, min_day = 260, max_day = 2600, horizon = 1):
    data = data.dropna()
    if len(data) < min_day:
        print('note enough data')
        return 0
    else:
        cov = []
        for i in range(len(data)):
            if i < (max_day - min_day):
                new = covariance_vol_decay(data.ix[:i+min_day], horizon)
            else:
                new = covariance_vol_decay(data.ix[(i+min_day-max_day):i+min_day], horizon)
            cov.append(new)
        cov = pd.Series(cov, index = data.index)
    return cov

In [22]:
N_cov_mat_vol_decay = rolling_cov_vol_decay(raw_data)

In [23]:
N_cov_mat_vol_decay[0]

,FX_MXN,FX_SGD,FX_ZAR,FX_INR,FX_BRL,FX_TRY
FX_MXN,1.958E-06,-1.096E-06,-16.989E-06,-1.565E-06,-9.810E-06,-16.691E-06
FX_SGD,-1.096E-06,613.831E-09,9.512E-06,876.110E-09,5.492E-06,9.345E-06
FX_ZAR,-16.989E-06,9.512E-06,147.395E-06,13.576E-06,85.109E-06,144.812E-06
FX_INR,-1.565E-06,876.110E-09,13.576E-06,1.250E-06,7.839E-06,13.338E-06
FX_BRL,-9.810E-06,5.492E-06,85.109E-06,7.839E-06,49.144E-06,83.618E-06
FX_TRY,-16.691E-06,9.345E-06,144.812E-06,13.338E-06,83.618E-06,142.275E-06


# 4

In [24]:
def log_likehood(data, cov_mat,lag):
    ans = 0
    data = data.dropna() # Need further calibration about the index
    u = data.mean(axis = 0)
    for x in range(lag,len(data)):
        temp = -0.5*(np.log(np.linalg.det(cov_mat.ix[x-lag]))+ \
                     np.dot(np.dot((data.ix[x]-u).T,np.mat(cov_mat.ix[x-lag]).I),\
                            (data.ix[x]-u))+6*np.log(2*np.pi))
        ans +=temp.item()
    return ans

# 6

In [143]:
def covariance_vol_corr_decay(data, vol_decay=0.1, corr_decay=0.1, horizon=1):
    names = list(data.columns.values)
    time = np.arange(len(data))[::-1]
    decay_vol = np.power((1 - vol_decay), time)
    decay_corr = np.power((1 - corr_decay), time)
    X = np.column_stack([data])
    X -= X.mean(axis=0) 
    cov = np.dot((X.T * decay_corr), X.conj()) / decay_corr.sum()
    cov_vol = np.dot((X.T * decay_vol), X.conj()) / decay_vol.sum()
    np.fill_diagonal(cov, np.diagonal(cov_vol))
    if horizon == 260:
        cov = cov * 260
    cov = pd.DataFrame(cov, index=names, columns=names)
    return cov

def rolling_cov_vol_corr_decay(data, vol_decay=0.1, corr_decay=0.1, min_day = 260, max_day = 2600, horizon = 1):
    data = data.dropna()
    if len(data) < min_day:
        print('note enough data')
        return 0
    else:
        cov = []
        for i in range(len(data)):
            if i < (max_day - min_day):
                new = covariance_vol_corr_decay(data.ix[:i+min_day], horizon)
            else:
                new = covariance_vol_corr_decay(data.ix[(i+min_day-max_day):i+min_day], horizon)
            cov.append(new)
        cov = pd.Series(cov, index = data.index)
    return cov

In [144]:
N_cov_mat_cov_vol_decay = rolling_cov_vol_corr_decay(raw_data)

In [145]:
N_cov_mat_cov_vol_decay[0]

,FX_MXN,FX_SGD,FX_ZAR,FX_INR,FX_BRL,FX_TRY
FX_MXN,1.958179e-06,-2.677706e-07,-0.000011,-1.560768e-07,0.000007,-2.562177e-07
FX_SGD,-2.677706e-07,6.138309e-07,0.000004,2.325848e-07,-0.000001,-8.543529e-09
FX_ZAR,-1.123044e-05,3.694668e-06,0.000147,2.108137e-06,-0.000013,6.248533e-05
FX_INR,-1.560768e-07,2.325848e-07,0.000002,1.250456e-06,0.000004,-1.095843e-07
FX_BRL,7.153219e-06,-1.323966e-06,-0.000013,3.532273e-06,0.000049,-2.466988e-05
FX_TRY,-2.562177e-07,-8.543529e-09,0.000062,-1.095843e-07,-0.000025,1.422747e-04
